In [1]:
import pandas as pd
from autogluon.tabular import TabularPredictor

/Users/ganxing/anaconda3/envs/ai/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 加载数据
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

In [3]:
# 删除Summary列，因为文本太多特征提取与分析耗时太长收益不高
# 删除Summary列和Id列
train_data = train_data.drop(columns=['Id'])

In [4]:
# 定义预测目标列名
label = 'SalePrice'

# 删除目标列中的空值行
train_data = train_data.dropna(subset=[label])

In [10]:
# AutoGluon训练，加入多模态配置
predictor = TabularPredictor(label=label).fit(
    train_data,
    presets=['best_quality'] 
)

No path specified. Models will be saved in: "AutogluonModels/ag-20240318_070358"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Dynamic stacking is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
Detecting stacked overfitting by sub-fitting AutoGluon on the input data. That is, copies of AutoGluon will be sub-fit on subset(s) of the data. Then, the holdout validation data is used to detect stacked overfitting.
Sub-fit(s) time limit is: 3600 seconds.
Starting holdout-based sub-fit for dynamic stacking. Context path is: AutogluonModels/ag-20240318_070358/ds_sub_fit/sub_fit_ho.
Running the sub-fit in a ray process to avoid memory leakage.
Spend 912 seconds for the sub-fit(s) during dynamic stacking.
Time left for full fit of AutoGluon: 2688 seconds.
Starting full fit now with num_stack_le

KeyboardInterrupt: 

In [7]:
# 预测测试集
predictions = predictor.predict(test_data)

In [9]:
# 把预测结果作为新列添加到测试数据的副本中
test_data_with_predictions = test_data.copy()
test_data_with_predictions[label] = predictions

# 仅保留Id和预测结果列
submission = test_data_with_predictions[['Id', label]]

# 保存预测结果到CSV文件，以符合提交格式
submission.to_csv('final_predictions.csv', index=False)